In [1]:
DATASET_PATH = "/home/choice/Desktop/Algorithmic Trading/algorithms/data/EURUSD_Candlestick_1_Hour_BID_01.07.2020-15.07.2023.csv"

## EDA

In [2]:
import pandas as pd

df = pd.read_csv(DATASET_PATH)

df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

### Checking for null values

In [3]:
df.isnull().sum()

# Removing values with Volume 0 where the market was closed
df = df[df['Volume'] != 0]

df.reset_index(drop=True, inplace=True)

In [4]:
df

,Time,Open,High,Low,Close,Volume
0,01.07.2020 00:00:00.000,1.12336,1.12336,1.12275,1.12306,4148.0298
1,01.07.2020 01:00:00.000,1.12306,1.12395,1.12288,1.12385,5375.5801
2,01.07.2020 02:00:00.000,1.12386,1.12406,1.12363,1.12382,4131.6099
3,01.07.2020 03:00:00.000,1.12382,1.12388,1.12221,1.12265,4440.6001
4,01.07.2020 04:00:00.000,1.12265,1.12272,1.12151,1.12179,4833.1001
...,...,...,...,...,...,...
17763,14.07.2023 16:00:00.000,1.12356,1.12430,1.12330,1.12394,10580.8500
17764,14.07.2023 17:00:00.000,1.12395,1.12395,1.12265,1.12340,10621.8000
17765,14.07.2023 18:00:00.000,1.12341,1.12366,1.12315,1.12340,11268.2900
17766,14.07.2023 19:00:00.000,1.12340,1.12340,1.12258,1.12259,7467.4400


## Pattern Detection

### Checking pivot id

This function checks if the current candle is a pivot high or a pivot low

In [5]:
def pivotid(df, index, n1, n2):
    """
    Description:
        - This function checks if a candle with given index is a pivot high or pivot low.

    Paramns:
        - df (DataFrame): Dataframe with data. 
        - index (int): Index of the candle we want to test for pivot low or pivot high
        - n1 (int): Number of neighbours to check with before the candle. 
        - n2 (int): Number of neighbours to check with after the candle. 

        If the candle is higher than previous n1 neighbours and following n2 neighbours, it is usually a pivot high. Same for pivot low. 

    returns:
        - 1 - If it is pivot low, 
        - 2 - If it is pivot high. 
        - 3 - If it is high and low at the same time
        - 0 - In any other case.  
    """

    # Check if previous n1 and following n2 is in dataframe. 
    if index - n1 < 0 or index + n2 >= len(df):
        return 0
    
    piv_id_low = 1
    piv_id_high = 1

    for i in range(index - n1, index + n2 + 1):
        if (df.Low[index] > df.Low[i]):
            piv_id_low = 0
        if (df.High[index] < df.High[i]):
            piv_id_high = 0
    
    if piv_id_high and piv_id_low:
        return 3
    elif piv_id_low:
        return 1
    elif piv_id_high:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name, 3, 3), axis=1)

In [6]:
df.head(5)

,Time,Open,High,Low,Close,Volume,pivot
0,01.07.2020 00:00:00.000,1.12336,1.12336,1.12275,1.12306,4148.0298,0
1,01.07.2020 01:00:00.000,1.12306,1.12395,1.12288,1.12385,5375.5801,0
2,01.07.2020 02:00:00.000,1.12386,1.12406,1.12363,1.12382,4131.6099,0
3,01.07.2020 03:00:00.000,1.12382,1.12388,1.12221,1.12265,4440.6001,0
4,01.07.2020 04:00:00.000,1.12265,1.12272,1.12151,1.12179,4833.1001,1


### Pivot Candles Visualization

In [7]:
import numpy as np

def pointpos(row):
    """
    Description:
        - This function visualizes the candles along with pivot points. 

    params:
        - row (dataframe): Row of a dataframe
    
    returns:
        - Pivot points
    """

    if row['pivot'] == 1:
        return row['Low']
    elif row['pivot'] == 2:
        return row['High']
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [8]:
! pip install plotly==5.22.0

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install nbformat 

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[500:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

### Detect flag function

In [26]:
from scipy.stats import linregress

def detect_flag(candle, backcandles, window, plot_flag=False):
    """
    Description:
        - This function  detects the flag

    parameters:
        - candle: The current candle in question
        - backcandles: Previous candles that need to be studied for the formation
        - window: Slice under study, the window should be greater than range(n1, n2)

    returns:
        - Flag
    """

    localdf = df[candle - backcandles - window: candle - window]

    # We take the most recent 3 highs
    highs = localdf[localdf['pivot'] == 2].High.tail(3).values
    indexhighs = localdf[localdf['pivot'] == 2].High.tail(3).index

    # We take the most recent 3 lows
    lows = localdf[localdf['pivot'] == 1].Low.tail(3).values
    indexlows = localdf[localdf['pivot'] == 1].Low.tail(3).index

    # Now we need to check if they are in an order -> Low High Low High or High Low High Low
    order_condition = ( 
            (indexlows[0] < indexhighs[0] 
            < indexlows[1] < indexhighs[1] 
            < indexlows[2] < indexhighs[2]) 
            or 
            (indexhighs[0] < indexlows[0] 
             < indexhighs[1] < indexlows[1] 
             < indexhighs[2] < indexlows[2]) )
    

    slmin, intercmin, rmin, _, _ = linregress(indexlows, lows)
    slmax, intercmax, rmax, _, _ = linregress(indexhighs, highs)

    # Ideal condition
    if (order_condition 
            and (rmax*rmax)>=0.9 
            and (rmin*rmin)>=0.9 
            and slmin>=0.0001 
            and slmax<=-0.0001):
        if plot_flag:
            fig = go.Figure(data=[go.Candlestick(
                x = localdf.index, 
                open = localdf['Open'], 
                high = localdf['High'], 
                low = localdf['Low'], 
                close = localdf['Close']
            )])

            fig.add_scatter(x=localdf.index, y=localdf['pointpos'], mode="markers",
            marker=dict(size=10, color="MediumPurple"),
            name="pivot")
            fig.add_trace(go.Scatter(x=indexlows, y=slmin*indexlows + intercmin, mode='lines', name='min slope'))
            fig.add_trace(go.Scatter(x=indexhighs, y=slmax*indexhighs + intercmax, mode='lines', name='max slope'))
            fig.update_layout(
            xaxis_rangeslider_visible=False,
            plot_bgcolor='white', # change the background to white
            xaxis=dict(showgrid=True, gridcolor='white'), # change the x-axis grid to white
            yaxis=dict(showgrid=True, gridcolor='white') # change the y-axis grid to white
                )
            fig.show()

        return 1
    return 0

In [57]:
flag_present = detect_flag(2189, 30, 3, plot_flag=True)

print(flag_present)

1


In [45]:
df[df['pivot'] == 2]

,Time,Open,High,Low,Close,Volume,pivot,pointpos
8,01.07.2020 08:00:00.000,1.12315,1.12448,1.12290,1.12311,10042.7695,2,1.12448
14,01.07.2020 14:00:00.000,1.12391,1.12751,1.12364,1.12605,20865.6504,2,1.12751
32,02.07.2020 08:00:00.000,1.12807,1.13026,1.12753,1.12972,12125.5996,2,1.13026
49,03.07.2020 01:00:00.000,1.12400,1.12492,1.12372,1.12423,3733.8501,2,1.12492
55,03.07.2020 07:00:00.000,1.12401,1.12468,1.12242,1.12350,11675.7998,2,1.12468
...,...,...,...,...,...,...,...,...
17703,12.07.2023 04:00:00.000,1.10295,1.10368,1.10292,1.10361,7046.7600,2,1.10368
17718,12.07.2023 19:00:00.000,1.11382,1.11403,1.11333,1.11370,9856.5800,2,1.11403
17750,14.07.2023 03:00:00.000,1.12353,1.12429,1.12335,1.12413,13061.2400,2,1.12429
17756,14.07.2023 09:00:00.000,1.12312,1.12448,1.12261,1.12267,14082.5700,2,1.12448
